In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

In [2]:
Number_of_project = 1000

In [3]:
def Function_to_choose(choice, list_projects):
    projects = [project for project in list_projects for _ in range(6)]
    cost = np.full(choice.shape[:1] + (len(projects),), Number_of_project)
    for no_students, range_choice_students in enumerate(choice):
        for no_choice_students, choice_students in enumerate(range_choice_students):
            if choice_students == 'CLOSED!':
                continue
            if choice_students == 'NAN':
                for no_project in range(len(projects)):
                    if cost[no_students, no_project] == Number_of_project:
                        cost[no_students, no_project] = function_no_choice(no_choice_students)
                break
            for no_project, project in enumerate(projects):
                if choice_students == project:
                    cost[no_students, no_project] = function_no_choice(no_choice_students)
    return cost

def close_project(project):
    df_choice[df_choice == project] = 'CLOSED!'
    list_projects.remove(project)

def function_no_choice(no_choice):
    if no_choice in [0, 1]:
        return no_choice
    if no_choice==2:
        return 5
    if no_choice==3:
        return 15
    if no_choice==4:
        return 25

In [4]:
df_choice = pd.read_excel('affectations.xlsx', 'affectation étudiants').dropna(0, 'all')
df_choice = df_choice[['Nom', 'Prénom', 'Choix 1 : référence', 'Choix 2 : Référence', 'Choix 3 : Référence',
               'Choix 4 : Référence', 'Choix 5 : Référence']]
df_choice.columns = ['Nom', 'Prénom', '1', '2', '3', '4', '5']
df_choice = df_choice.astype(str)
df_choice[['1', '2', '3', '4', '5']] = df_choice[['1', '2', '3', '4', '5']].applymap(lambda elt: elt.upper())

list_projects = list(pd.read_excel('affectations.xlsx', 'Choix projet effectif')['Numéro du projet'].dropna())

df_choice[~df_choice[['1', '2', '3', '4', '5']].isin(list_projects)] = 'NAN'

for no_students, range_choice_students in enumerate(df_choice.values[:,-5:]):
    projects_choices = list()
    for choice_students in range_choice_students:
        if choice_students != 'NAN' and choice_students not in projects_choices:
            projects_choices.append(choice_students)
    for _ in range(5 - len(projects_choices)):
        projects_choices.append('NAN')
    df_choice.iloc[no_students, -5:] = projects_choices

In [5]:
while True:
    cost = Function_to_choose(df_choice.values[:,-5:], list_projects)
    _, affectations = linear_sum_assignment(cost)
    affectations = affectations // 6
    count_affectations = {project:0 for project in list_projects}
    for affectation in affectations:
        count_affectations[list_projects[affectation]] += 1
    project_min_affectations = min(count_affectations, key=count_affectations.get)
    if count_affectations[project_min_affectations] >= 4:
        break
    close_project(project_min_affectations)

In [6]:
choice_shown = np.array(list_projects)[[affectation for affectation in affectations for _ in range(5)]]
choice_shown = choice_shown.reshape(len(df_choice), 5)
choice_shown = (df_choice.values[:,-5:] == choice_shown).astype('<U24')
choice_shown[choice_shown == 'True'] = 'background-color: yellow'
choice_shown[choice_shown == 'False'] = ''
choice_shown = np.concatenate((np.full((len(df_choice),2),'').astype('<U24'), choice_shown), axis=1)
choice_shown = pd.DataFrame(choice_shown.tolist())
choice_shown.columns = df_choice.columns
df_choice.style.apply(lambda tab: choice_shown, axis=None)

,Nom,Prénom,1,2,3,4,5
0,BRIERRE-BATTAIS,Tim,C1,CLOSED!,CLOSED!,R5,CLOSED!
1,ESPINASSE,Baptiste,R5,C1,R16,CLOSED!,R29
2,JOUNIAUX,Colin,C1,C2,R7,CLOSED!,R5
3,MEMMO,Kevin,C2,C1,R18,E16,CLOSED!
4,PININGRE,Alban,C1,R24,E10,R12,CLOSED!
5,GOEURIOT,Damien,CLOSED!,C1,E16,C2,E3
6,MENAUD,Adrien,C1,NAN,NAN,NAN,NAN
7,NATKUNARAJAH,Ninthija,R32,R24,C1,E10,CLOSED!
8,BERNARDINI,Baptiste,C3,CLOSED!,C1,R3,R22
9,BOISSEVAL,Sébastien,C3,CLOSED!,R18,R22,R29
